# KMP 算法

> 用于在文本中匹配给定的子串（模式串）
> 假设我们现在有这样一个问题：有一个文本串S，一个模式串P，问P出现在S中吗？如果出现，P在S的什么位置？

## 暴力算法

> 遍历，如果当前遍历到文本位置i，模式串位置j，如果 S[i] == P[j], 则 i+=1, j+=1 继续匹配下一个位置；否则 i = i - (j-1), j = 0，相当于模式串右移一位，从模式串头对齐的位置匹配

In [20]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [16]:
# brute force solution
def violent_match(s, p):
    s_len = len(s)
    p_len = len(p)
    i = 0
    while i <= s_len - p_len:
        j = 0
        while j < p_len and s[i] == p[j]:
            i += 1
            j += 1
        
        if j == p_len:
            return i - j
        else:
            i = i - j + 1
    return -1

violent_match("hello", "ll")

2

### 暴力算法的缺点

> 当 S[i] != P[j] 时，暴力算法回退比较 S[i-j+1] 和 P[0]，之前已经知道S[i-j+1] = P[1] (假设 j > 1), 所以等同于比较 P[0] 和 P[1]，这是已知的信息，有没有什么办法利用模式串的信息来优化算法呢

## RK 算法

> 计算主串中每个子串的hash值（子串的hash值可以从前一个子串的hash值中迭代计算），和模式串的hash比较



In [15]:
# 假设要匹配的字符集是a～z，hash函数逻辑为ord(s[0]) + ord(s[1]) * 26 + ord(s[2]) * 26^2 + ... 


def rk(s, p):
    n = len(s)
    m = len(p)
    factor = [ 26**i for i in range(m)]
    
    def hash(text):
        l = len(text)
        return sum((ord(text[i]) - ord('a')) * factor[l-i-1]  for i in range(l))

    p_hash = hash(p)
    sub_hash = hash(s[:m])
    i = 0
    
    while sub_hash != p_hash and i < n - m:
        sub_hash = (sub_hash - hash(s[i]) * factor[-1]) * factor[1] + hash(s[i+m])
        i += 1

    return -1 if sub_hash != p_hash else i

rk('hello', 'll')

2

## KMP 算法

> 这里先对 KMP 的流程进行概述，然后再详细解释

> 1. 当 S[i] == P[j] 时，比较S[++i] 和 P[++j]
> 2. 当 S[i] != P[j] 时，比较S[++i] 和 P[next[j-1]]（j>0），或者比较S[++i] 和 P[j]（j==0）
> 3. 当 j == p_len 时，在S中找到模式串的位置i - j, 匹配结束
> 4. 当 i == s_len 时，无匹配，匹配结束


### next 数组

> 不同的实现方式有不同的定义，这里是其中一种定义

> 定义：next 数组的值是对应的子串（比如下标i，对应P[:i+1]）的前缀、后缀中公共元素的最大长度（下文简称最大长度）


### 字符串的前缀、后缀

> 前缀：不包括尾字符，所有包括头字符的子串组合
> 后缀：不包括头字符，所有包括尾字符的子串组合

#### 举个例子
> 字符串：abcab
> 前缀：[abca, abc, ab, a]
> 后缀：[b, ab, cab, bcab]

> next数组
> 1. next[0] 对应的子串是：a，最大长度是：0
> 2. next[1] 对应的子串是：ab，最大长度是：0
> 3. next[2] 对应的子串是：abc，最大长度是：0
> 4. next[3] 对应的子串是：abca，最大长度是：1
> 5. next[4] 对应的子串是：abcab，最大长度是：2


#### next数组实现

> 1. next[0] = 0, j=0（记录前缀的位置），i=1（记录后缀的位置）
> 2. 如果 j>0 且 P[j] != P[i], j=next[j-1]
> 3. 如果 P[j] == P[i], j++
> 4. next[i] = j

In [17]:
# next数组实现
def build_next(p):
    l = len(p)
    next = [0] * l
    j = 0
    for i in range(1, l):
        while j > 0 and p[i] != p[j]:
            j = next[j - 1]
        if p[i] == p[j]:
            j += 1
        next[i] = j
    return next

build_next("ababaca")

[0, 0, 1, 2, 3, 0, 1]

In [20]:
# KMP算法实现
def kmp(s, p):
    s_len = len(s)
    p_len = len(p)
    next = build_next(p)
    j = 0
    for i in range(s_len):
        while j > 0 and s[i] != p[j]:
            j = next[j - 1]
        if s[i] == p[j]:
            j += 1
        if j == p_len:
            return i - j + 1
    return -1

kmp("hello", "ll")

2

## 扩展

### BM算法

> 用于在文本S中搜索模式串P
>
> [参考](https://writings.sh/post/algorithm-string-searching-boyer-moore)
>
> [参考](image/字符串匹配%20-%20Boyer–Moore%20算法原理和实现.pdf)



#### 整体思路
> 1. 从模式串P尾部开始匹配，i, j 分别为 S、P 当前比较的位置下标，n, m分别为 S、P的长度，初始化 i=j=m-1
> 2. 当 S[i] == P[j], i--, j--
> 3. 当 S[i] != P[j], S[i] 称为“坏字符”，S[i+1:i+m-j]（后面已经匹配上的子串）称为“好后缀”。例如S为ABCAD，P为BAD，当 i = 2, j = 0时, “C”为坏字符，“AD”为好后缀
> 4. 模式串有两种向后移动的方法(规则)
> 5. 坏字符规则：在P当前失配位置左边找最近的匹配字符。例如P为ABCAB，失配时，S[i] = A, j=4, 最近的匹配字符为j=3的”A“
> 6. 好后缀规则：在P当前失配位置左边找最近的匹配子串。例如P为ABCAB，失配时，S[i] = C, j=2，最近的匹配子串为j=0的”AB“
> 7. 当 i >= s_len, 没匹配；当j < 0, 匹配

##### 坏字符规则

> 后移模式串，等同于后移i，也就是我们要找出i后移的距离d
> 二维坏字符表：行表示文本串中的字符，列表示失配时在P的位置，值表示i需要移动的距离d，d > m - j，坏字符在左边的位置为k，d = m - k

In [18]:
# 2D bad character table
import numpy as np
import pandas as pd

def bad_character_table(p):
    m = len(p)
    char = list(set(p))
    table = pd.DataFrame(np.full((len(char), m), m), index=char)
    for i in range(1, m):
        table.iloc[:, i] = table.iloc[:, i-1]
        table.loc[p[i-1], i] = m - i
    return table

bad_character_table('BCDBACD')

,0,1,2,3,4,5,6
B,7,6,6,6,3,3,3
D,7,7,7,4,4,4,4
C,7,7,5,5,5,5,1
A,7,7,7,7,7,2,2


##### 好后缀规则

> 分两种情况
> 1. 失配位置的左边能找到好后缀，假设失配位置为j，相当于找P[:j]最长的子串（最长前缀串），子串后缀为P[j+1:]，i移动的距离d，d > m-j, 重复串头部位置为t，d = m - t
> 2. 失配位置的左边不能找到好后缀或能在前缀中找到部分好后缀，d = m + len(suffix) - len(prefix)

> 因为我们要在失配位置左边找离失配位置最近的好后缀，所以先找第二种情况，再找第一种情况

In [22]:
# good suffix table
def good_suffix_table(p):
    m = len(p)
    table = [0] * m
    # suffix not found
    def good_suffix_not_on_left(p):
        table[m-1] = 1
        last_prefix_length = 0
        for i in range(m-1, 0, -1):
            suffix_length = m - i
            if p[i:] == p[:m-i]:
                last_prefix_length = prefix_length = suffix_length
            else:
                prefix_length = last_prefix_length
            table[i-1] = m + suffix_length - prefix_length
    # suffix found
    def good_suffix_on_left(p):
        def common_suffix_length(p, i, j):
            length = 0
            while i >= 0 and p[i] == p[j]:
                length += 1
                i -= 1
                j -= 1
            return length
        
        for i in range(m-1):
            common_suffix_length_ = common_suffix_length(p, i, m-1)
            if common_suffix_length_ > 0:
                j = m - 1 - common_suffix_length_
                table[j] = m - (i + 1 - common_suffix_length_)
    
    good_suffix_not_on_left(p)
    good_suffix_on_left(p)
    return table

> 结合“坏字符”规则和“好后缀”规则，取两者后移比较大的值

> 实践中，经常用一维”坏字符“规则，原因是实现简单，且结合上述逻辑，不会出现死循环

In [21]:
# 1D bad character table
def bad_character_table(p):
    m = len(p)
    char = list(set(p))
    table = pd.Series([m] * len(char), index=char)
    table[p[-1]] = 1
    for i in range(m-1):
        table[p[i]] = m - 1 - i
    return table

char_table = bad_character_table('BCDBACD')
char_table
char_table.get('F', len('BCDBACD'))

B    3
D    4
C    1
A    2
dtype: int64

7

In [ ]:
# BM
def bm(s, p):
    n, m = len(s), len(p)
    if n < m:
        return -1
    bad_char = bad_character_table(p)
    good_suffix = good_suffix_table(p)
    i = m - 1
    while i < n:
        j = m - 1
        while j >= 0 and s[i] == p[j]:
            i -= 1
            j -= 1
        if j == -1:
            return i + 1
        i += max(bad_char.get(s[i], m), good_suffix[j]) # max(bad_char.loc[s[i], j], good_suffix[j])
    return -1

bm('BCBAABACAABABACAA', 'ABABAC')

##### 复杂度分析

> “坏字符“规则：一维是O(S + M)，二维是O(S * M), S是考虑的字符集，M是P的长度
> “好后缀”规则：O(M2)
> 查找：O(N*M)

> 使用场景：M远比N小